# Laboratorio no calificado: Recorrido por los metadatos de ML

Mantener registros en cada etapa del proyecto es un aspecto importante de los conductos de aprendizaje automático. Especialmente en los modelos de producción que implican muchas iteraciones de conjuntos de datos y reentrenamiento, tener estos registros ayudará a mantener o depurar el sistema desplegado. [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) responde a esta necesidad al disponer de una API adaptada específicamente para el seguimiento de los progresos realizados en los proyectos de ML.

Como se ha mencionado en los laboratorios anteriores, ya has utilizado ML Metadata cuando has ejecutado tus pipelines TFX. Cada componente registra automáticamente la información en un almacén de metadatos a medida que va pasando por cada etapa. Esto le permitió recuperar información como el nombre de las divisiones de entrenamiento o la ubicación de un esquema inferido. 

En este cuaderno, se estudiará más detenidamente cómo se pueden utilizar los metadatos ML directamente para registrar y recuperar metadatos independientemente de un pipeline TFX (es decir, sin utilizar componentes TFX). Utilizará TFDV para inferir un esquema y registrar toda la información sobre este proceso. Esto mostrará cómo se relacionan los diferentes componentes entre sí para que puedas interactuar mejor con la base de datos cuando vuelvas a usar TFX en los próximos laboratorios. Además, conocer el funcionamiento interno de la librería te ayudará a adaptarla a otras plataformas si es necesario.

¡Vamos a ello!

In [1]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))

import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

import urllib
import zipfile

TF version: 2.6.0
TFDV version: 1.3.0


## Descargar conjunto de datos

Para este laboratorio utilizará el conjunto de datos [Chicago Taxi](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew). Vamos a descargar los CSV en su espacio de trabajo.

In [2]:
# Descargue el archivo zip de GCP y descomprímalo
url = 'https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip'
zip, headers = urllib.request.urlretrieve(url)
zipfile.ZipFile(zip).extractall()
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R data

Here's what we downloaded:
data:
census_data  eval  serving  train

data/census_data:
adult.data

data/eval:
data.csv

data/serving:
data.csv

data/train:
data.csv


## Esquema del proceso

A continuación se muestra la figura mostrada en clase que describe los diferentes componentes en un almacén de Metadatos ML:

<img src='img/mlmd_overview.png' alt='imagen de la vista general de mlmd'>

El recuadro verde del centro muestra el modelo de datos seguido de los Metadatos ML. La [documentación oficial](https://www.tensorflow.org/tfx/guide/mlmd#data_model) describe cada uno de ellos y lo mostraremos aquí también para facilitar la referencia:

* `ArtifactType` describe el tipo de un artefacto y sus propiedades que se almacenan en el almacén de metadatos. Puedes registrar estos tipos sobre la marcha en el almacén de metadatos en código, o puedes cargarlos en el almacén desde un formato serializado. Una vez registrado un tipo, su definición está disponible durante toda la vida del almacén.
* Un `Artifact` describe una instancia específica de un ArtifactType, y sus propiedades que se escriben en el almacén de metadatos.
* Un "Tipo de Ejecución" describe un tipo de componente o paso en un flujo de trabajo, y sus parámetros de ejecución.
* Una "Ejecución" es un registro de la ejecución de un componente o un paso en un flujo de trabajo de ML y los parámetros de tiempo de ejecución. Una ejecución puede ser considerada como una instancia de un ExecutionType. Las ejecuciones se registran cuando se ejecuta una tubería o un paso de ML.
* Un `Evento` es un registro de la relación entre artefactos y ejecuciones. Cuando ocurre una ejecución, los eventos registran cada artefacto que fue utilizado por la ejecución, y cada artefacto que fue producido. Estos registros permiten el seguimiento del linaje a través de un flujo de trabajo. Al mirar todos los eventos, MLMD sabe qué ejecuciones ocurrieron y qué artefactos se crearon como resultado. MLMD puede entonces recurrir desde cualquier artefacto a todas sus entradas anteriores.
* Un `ContextType` describe un tipo de grupo conceptual de artefactos y ejecuciones en un flujo de trabajo, y sus propiedades estructurales. Por ejemplo: proyectos, ejecuciones de pipeline, experimentos, propietarios, etc.
* Un "contexto" es una instancia de un tipo de contexto. Captura la información compartida dentro del grupo. Por ejemplo: el nombre del proyecto, el identificador de la lista de cambios, las anotaciones de los experimentos, etc. Tiene un nombre único definido por el usuario dentro de su ContextType.
* Una "Atribución" es un registro de la relación entre artefactos y contextos.
* Una "Asociación" es un registro de la relación entre ejecuciones y contextos.

Como se mencionó anteriormente, se utilizará TFDV para generar un esquema y registrar este proceso en el almacén de metadatos de ML. Empezarás desde cero, por lo que definirás cada componente del modelo de datos. El esquema de pasos implica:

1. Definir la base de datos de almacenamiento de ML Metadata
1. Configurar los tipos de artefactos necesarios
1. Configuración de los tipos de ejecución
1. Generación de una unidad de artefacto de entrada
1. Generación de una unidad de ejecución
1. Registro de un evento de entrada
1. Ejecución del componente TFDV
1. Generar una unidad de artefacto de salida
1. Registro de un evento de salida
1. Actualizar la unidad de ejecución
1. Configuración y generación de una unidad de contexto
1. Generación de atribuciones y asociaciones

A continuación, puede recuperar información de la base de datos para investigar aspectos de su proyecto. Por ejemplo, puede encontrar qué conjunto de datos se utilizó para generar un esquema concreto. También lo hará en este ejercicio.

Para cada uno de estos pasos, es posible que quieras tener la [documentación de la API de MetadataStore](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore) abierta para que puedas buscar cualquiera de los métodos que vas a utilizar para interactuar con el almacén de metadatos. También puedes consultar el buffer del protocolo `metadata_store` [aquí](https://github.com/google/ml-metadata/blob/r0.24.0/ml_metadata/proto/metadata_store.proto) para ver las descripciones de cada tipo de datos que se tratan en este tutorial.

## Definir la base de datos de almacenamiento de ML Metadata

El primer paso sería instanciar su backend de almacenamiento. Como se mencionó en la clase, hay varios tipos soportados como la base de datos falsa (temporal), SQLite, MySQL, e incluso el almacenamiento basado en la nube. Para esta demostración, sólo se utilizará una base de datos falsa para una rápida experimentación.

In [3]:
# Instanciar una conexión config
connection_config = metadata_store_pb2.ConnectionConfig()

# Establecer una base de datos falsa proto vacia 
connection_config.fake_database.SetInParent() 

# Configurar el almacén de metadatos
store = metadata_store.MetadataStore(connection_config)

## Registrar tipos de artefactos

A continuación, crearemos los tipos de artefactos necesarios y los registraremos en el almacén. Dado que nuestro sencillo ejercicio sólo implicará la generación de un esquema utilizando TFDV, sólo crearás dos tipos de artefactos: uno para el **conjunto de datos de entrada** y otro para el **esquema de salida**. Los pasos principales serán:

* Declarar un `ArtifactType()`.
* Definir el nombre del tipo de artefacto
* Definir las propiedades necesarias dentro de estos tipos de artefactos. Por ejemplo, es importante conocer el nombre de la división de los datos, por lo que puede querer tener una propiedad `split` para el tipo de artefacto que contiene conjuntos de datos.
* Usa `put_artifact_type()` para registrarlos en el almacén de metadatos. Esto genera un `id` que puedes usar más tarde para referirte a un tipo de artefacto en particular.

*Bonus: Para practicar, también puedes extender el código de abajo para crear un tipo de artefacto para las estadísticas.*

In [4]:
# Crear ArtifactType para el conjunto de datos de entrada
data_artifact_type = metadata_store_pb2.ArtifactType()
data_artifact_type.name = 'DataSet'
data_artifact_type.properties['nombre'] = metadata_store_pb2.STRING
data_artifact_type.properties['split'] = metadata_store_pb2.STRING
data_artifact_type.properties['version'] = metadata_store_pb2.INT

# Registrar el tipo de artefacto en el almacén de metadatos
data_artifact_type_id = store.put_artifact_type(data_artifact_type)

# Crear ArtifactType para el esquema
schema_artifact_type = metadata_store_pb2.ArtifactType()
schema_artifact_type.name = 'Schema'
schema_artifact_type.properties['nombre'] = metadata_store_pb2.STRING
schema_artifact_type.properties['version'] = metadata_store_pb2.INT

# Registrar el tipo de artefacto en el almacén de metadatos
schema_artifact_type_id = store.put_artifact_type(schema_artifact_type)

print('Data artifact type:\n', data_artifact_type)
print('Schema artifact type:\n', schema_artifact_type)
print('Data artifact type ID:', data_artifact_type_id)
print('Schema artifact type ID:', schema_artifact_type_id)

Data artifact type:
 name: "DataSet"
properties {
  key: "nombre"
  value: STRING
}
properties {
  key: "split"
  value: STRING
}
properties {
  key: "version"
  value: INT
}

Schema artifact type:
 name: "Schema"
properties {
  key: "nombre"
  value: STRING
}
properties {
  key: "version"
  value: INT
}

Data artifact type ID: 10
Schema artifact type ID: 11


## Register ExecutionType

A continuación, crearás los tipos de ejecución necesarios. Para la configuración simple, sólo declarará uno para el componente de validación de datos con una propiedad `state` para poder registrar si el proceso se está ejecutando o ya ha terminado.

In [5]:
# Crear ExecutionType para el componente de Validación de Datos
dv_execution_type = metadata_store_pb2.ExecutionType()
dv_execution_type.name = 'Data Validation'
dv_execution_type.properties['state'] = metadata_store_pb2.STRING

# Registrar el tipo de ejecución en el almacén de metadatos
dv_execution_type_id = store.put_execution_type(dv_execution_type)

print('Data validation execution type:\n', dv_execution_type)
print('Data validation execution type ID:', dv_execution_type_id)

Data validation execution type:
 name: "Data Validation"
properties {
  key: "state"
  value: STRING
}

Data validation execution type ID: 12


## Generar unidad de artefacto de entrada

Con los tipos de artefactos creados, ahora puede crear instancias de esos tipos. La celda de abajo crea el artefacto para el conjunto de datos de entrada. Este artefacto se registra en el almacén de metadatos a través de la función `put_artifacts()`. De nuevo, genera un `id` que puede ser utilizado como referencia.

In [7]:
# Declarar artefacto de entrada de tipo DataSet
data_artifact = metadata_store_pb2.Artifact()
data_artifact.uri = './data/train/data.csv'
data_artifact.type_id = data_artifact_type_id
data_artifact.properties['nombre'].string_value = 'Chicago Taxi dataset'
data_artifact.properties['split'].string_value = 'train'
data_artifact.properties['version'].int_value = 1

# Enviar el artefacto de entrada al almacén de metadatos
data_artifact_id = store.put_artifacts([data_artifact])[0]

print('Data artifact:\n', data_artifact)
print('Data artifact ID:', data_artifact_id)

Data artifact:
 type_id: 10
uri: "./data/train/data.csv"
properties {
  key: "nombre"
  value {
    string_value: "Chicago Taxi dataset"
  }
}
properties {
  key: "split"
  value {
    string_value: "train"
  }
}
properties {
  key: "version"
  value {
    int_value: 1
  }
}

Data artifact ID: 1


## Generar unidad de ejecución

A continuación, crearás una instancia del tipo de ejecución `Validación de Datos` que registraste anteriormente. Establecerás el estado a `RUNNING` para indicar que estás a punto de ejecutar la función TFDV. Esto se registra con la función `put_executions()`.

In [8]:
# Register the Execution of a Data Validation run
dv_execution = metadata_store_pb2.Execution()
dv_execution.type_id = dv_execution_type_id
dv_execution.properties['state'].string_value = 'RUNNING'

# Submit execution unit to the Metadata Store
dv_execution_id = store.put_executions([dv_execution])[0]

print('Data validation execution:\n', dv_execution)
print('Data validation execution ID:', dv_execution_id)

Data validation execution:
 type_id: 12
properties {
  key: "state"
  value {
    string_value: "RUNNING"
  }
}

Data validation execution ID: 1


## Registrar evento de entrada

Un evento define una relación entre artefactos y ejecuciones. Se generará la relación de eventos de entrada para las unidades de ejecución de conjuntos de datos y de validación de datos. La lista de tipos de eventos se muestra [aquí](https://github.com/google/ml-metadata/blob/master/ml_metadata/proto/metadata_store.proto#L187) y el evento se registra con la función `put_events()`.

In [9]:
# Declarar el evento de entrada
input_event = metadata_store_pb2.Event()
input_event.artifact_id = data_artifact_id
input_event.execution_id = dv_execution_id
input_event.type = metadata_store_pb2.Event.DECLARED_INPUT

# Submit input event to the Metadata Store
store.put_events([input_event])

print('Input event:\n', input_event)

Input event:
 artifact_id: 1
execution_id: 1
type: DECLARED_INPUT



## Ejecutar el componente TFDV

Ahora ejecutará el componente TFDV para generar el esquema del conjunto de datos. Esto debería resultarle familiar ya que lo ha hecho en la semana 1.

In [10]:
# Infer a schema by passing statistics to `infer_schema()`
train_data = './data/train/data.csv'
train_stats = tfdv.generate_statistics_from_csv(data_location=train_data)
schema = tfdv.infer_schema(statistics=train_stats)

schema_file = './schema.pbtxt'
tfdv.write_schema_text(schema, schema_file)

print("Dataset's Schema has been generated at:", schema_file)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset's Schema has been generated at: ./schema.pbtxt


## Generar unidad de artefacto de salida

Ahora que el componente TFDV ha terminado de ejecutarse y se ha generado el esquema, puede crear el artefacto para el esquema generado.

In [12]:
# Declare output artifact of type Schema_artifact
schema_artifact = metadata_store_pb2.Artifact()
schema_artifact.uri = schema_file
schema_artifact.type_id = schema_artifact_type_id
schema_artifact.properties['version'].int_value = 1
schema_artifact.properties['nombre'].string_value = 'Chicago Taxi Schema'

# Submit output artifact to the Metadata Store
schema_artifact_id = store.put_artifacts([schema_artifact])[0]

print('Schema artifact:\n', schema_artifact)
print('Schema artifact ID:', schema_artifact_id)

Schema artifact:
 type_id: 11
uri: "./schema.pbtxt"
properties {
  key: "nombre"
  value {
    string_value: "Chicago Taxi Schema"
  }
}
properties {
  key: "version"
  value {
    int_value: 1
  }
}

Schema artifact ID: 2


## Registrar evento de salida

De forma análoga al evento de entrada anterior, también se quiere definir un evento de salida para registrar el artefacto de salida de una unidad de ejecución concreta.

In [13]:
# Declare the output event
output_event = metadata_store_pb2.Event()
output_event.artifact_id = schema_artifact_id
output_event.execution_id = dv_execution_id
output_event.type = metadata_store_pb2.Event.DECLARED_OUTPUT

# Submit output event to the Metadata Store
store.put_events([output_event])

print('Output event:\n', output_event)

Output event:
 artifact_id: 2
execution_id: 1
type: DECLARED_OUTPUT



## Actualizar la unidad de ejecución

Como el componente TFDV ha terminado de ejecutarse con éxito, es necesario actualizar el `estado` de la unidad de ejecución y registrarlo de nuevo en el almacén.

In [14]:
# Mark the `state` as `COMPLETED`
dv_execution.id = dv_execution_id
dv_execution.properties['state'].string_value = 'COMPLETED'

# Update execution unit in the Metadata Store
store.put_executions([dv_execution])

print('Data validation execution:\n', dv_execution)

Data validation execution:
 id: 1
type_id: 12
properties {
  key: "state"
  value {
    string_value: "COMPLETED"
  }
}



## Configuración de tipos de contexto y generación de una unidad de contexto

Puede agrupar los artefactos y las unidades de ejecución en un `Contexto`. En primer lugar, es necesario definir un `ContextType` que defina el contexto requerido. Sigue un formato similar al de los tipos de artefactos y eventos. Puedes registrarlo con la función `put_context_type()`.

In [15]:
# Create a ContextType
expt_context_type = metadata_store_pb2.ContextType()
expt_context_type.name = 'Experiment'
expt_context_type.properties['note'] = metadata_store_pb2.STRING

# Register context type to the Metadata Store
expt_context_type_id = store.put_context_type(expt_context_type)

Del mismo modo, puede crear una instancia de este tipo de contexto y utilizar el método `put_contexts()` para registrarse en el almacén.

In [16]:
# Generate the context
expt_context = metadata_store_pb2.Context()
expt_context.type_id = expt_context_type_id
# Give the experiment a name
expt_context.name = 'Demo'
expt_context.properties['note'].string_value = 'Walkthrough of metadata'

# Submit context to the Metadata Store
expt_context_id = store.put_contexts([expt_context])[0]

print('Experiment Context type:\n', expt_context_type)
print('Experiment Context type ID: ', expt_context_type_id)

print('Experiment Context:\n', expt_context)
print('Experiment Context ID: ', expt_context_id)

Experiment Context type:
 name: "Experiment"
properties {
  key: "note"
  value: STRING
}

Experiment Context type ID:  13
Experiment Context:
 type_id: 13
name: "Demo"
properties {
  key: "note"
  value {
    string_value: "Walkthrough of metadata"
  }
}

Experiment Context ID:  1


## Generar relaciones de atribución y asociación

Con el `Contexto` definido, ahora puedes crear su relación con el artefacto y las ejecuciones que utilizaste previamente. Crearás la relación entre la unidad de artefacto del esquema y la unidad de contexto del experimento para formar una `Atribución`.
Del mismo modo, crearás la relación entre la unidad de ejecución de validación de datos y la unidad de contexto del experimento para formar una `Asociación`. Estas se registran con el método `put_attributions_and_associations()`.

In [17]:
# Generate the attribution
expt_attribution = metadata_store_pb2.Attribution()
expt_attribution.artifact_id = schema_artifact_id
expt_attribution.context_id = expt_context_id

# Generate the association
expt_association = metadata_store_pb2.Association()
expt_association.execution_id = dv_execution_id
expt_association.context_id = expt_context_id

# Submit attribution and association to the Metadata Store
store.put_attributions_and_associations([expt_attribution], [expt_association])

print('Experiment Attribution:\n', expt_attribution)
print('Experiment Association:\n', expt_association)

Experiment Attribution:
 artifact_id: 2
context_id: 1

Experiment Association:
 execution_id: 1
context_id: 1



## Recuperación de la información del almacén de metadatos

Ahora has registrado la información necesaria en el almacén de metadatos. Si hicimos esto en una base de datos persistente, puedes rastrear qué artefactos y eventos están relacionados entre sí incluso sin ver el código utilizado para generarlo. Vea un ejemplo de ejecución a continuación donde se investiga qué conjunto de datos se utiliza para generar el esquema. (**Es obvio qué conjunto de datos se utiliza en nuestra sencilla demostración porque sólo tenemos dos artefactos registrados. Por lo tanto, suponga que tiene miles de entradas en el almacén de metadatos*).

In [18]:
# Get artifact types
store.get_artifact_types()

[id: 10
 name: "DataSet"
 properties {
   key: "nombre"
   value: STRING
 }
 properties {
   key: "split"
   value: STRING
 }
 properties {
   key: "version"
   value: INT
 },
 id: 11
 name: "Schema"
 properties {
   key: "nombre"
   value: STRING
 }
 properties {
   key: "version"
   value: INT
 }]

In [19]:
# Obtener el primer elemento de la lista de artefactos `Schema`.
# Investigarás qué conjunto de datos se utilizó para generarlo.

schema_to_inv = store.get_artifacts_by_type('Schema')[0]

# print output
print(schema_to_inv)

id: 2
type_id: 11
uri: "./schema.pbtxt"
properties {
  key: "nombre"
  value {
    string_value: "Chicago Taxi Schema"
  }
}
properties {
  key: "version"
  value {
    int_value: 1
  }
}
create_time_since_epoch: 1667420946878
last_update_time_since_epoch: 1667420946878



In [20]:
# Get events related to the schema id
schema_events = store.get_events_by_artifact_ids([schema_to_inv.id])

print(schema_events)

[artifact_id: 2
execution_id: 1
type: DECLARED_OUTPUT
milliseconds_since_epoch: 1667421026595
]


Se ve que es una salida de una ejecución por lo que se puede buscar el id de la ejecución para ver los artefactos relacionados.

In [21]:
# Get events related to the output above
execution_events = store.get_events_by_execution_ids([schema_events[0].execution_id])

print(execution_events)

[artifact_id: 1
execution_id: 1
type: DECLARED_INPUT
milliseconds_since_epoch: 1667420834390
, artifact_id: 2
execution_id: 1
type: DECLARED_OUTPUT
milliseconds_since_epoch: 1667421026595
]


Verás la entrada declarada de esta ejecución para que puedas seleccionarla de la lista y buscar los detalles del artefacto.

In [23]:
# Look up the artifact that is a declared input
artifact_input = execution_events[0]

store.get_artifacts_by_id([artifact_input.artifact_id])

[id: 1
 type_id: 10
 uri: "./data/train/data.csv"
 properties {
   key: "nombre"
   value {
     string_value: "Chicago Taxi dataset"
   }
 }
 properties {
   key: "split"
   value {
     string_value: "train"
   }
 }
 properties {
   key: "version"
   value {
     int_value: 1
   }
 }
 create_time_since_epoch: 1667420711333
 last_update_time_since_epoch: 1667420711333]

Muy bien. Ahora has obtenido el artefacto del conjunto de datos que se utilizó para generar el esquema. Puedes abordar esto de otra manera y te instamos a que practiques utilizando los diferentes métodos de la API de MetadataStore para familiarizarte con la interacción con la base de datos.

### Recapitulación

En este cuaderno, has podido practicar el uso de ML Metadata fuera de TFX. Esto debería ayudarte a entender su funcionamiento interno para que sepas mejor cómo consultar los almacenes de ML Metadata o incluso configurarlo para tus propios casos de uso. TFX aprovecha esta biblioteca para mantener los registros de las ejecuciones del pipeline y podrás ver más de eso en los próximos laboratorios. A continuación, se revisará cómo trabajar con esquemas y en el próximo cuaderno, se verá cómo se puede implementar en una tubería TFX.